# `Step 4: Outlier`
- Free
- Z Score
- ZScore
- ZScore 

# 0- `Config`

In [ ]:
import pandas as pd 
import numpy as np 
import pickle 

In [ ]:
try: 
    if manager == 1:
        print("s1_load.ipynb running from MANAGER")
except: 
    %run s0_config.ipynb 
    subject_outlier_0 = "ZScore_v0"  
    subject_outlier_1 = "Empty_v0" 

# 1- `Load`

In [ ]:
import_name = "data_raw"
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_load}", f"{import_name}" + ".pkl")) , 'rb') as file:
    data_raw = pickle.load(file) 

import_name = "data_full"
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_load}", f"{import_name}" + ".pkl")) , 'rb') as file:
    data_full = pickle.load(file) 

---
# 2- `Preprocessing`

### 0-) `Overview`

In [ ]:
print(f"Subject Outlier of Embeddings: {subject_outlier_0}")
print(f"Subject Outlier of Expression: {subject_outlier_1}") 
print(f"Shape: {data_full.shape}") 
print(f"Number of unique cell types: {len(pd.unique(data_full['Cell_Type']))}")  
print(f"Unique cell types: {pd.unique(data_full['Cell_Type'])}")  
pd.concat([data_full.head(2), data_full.tail(2)], axis=0) 

In [ ]:
data_full["Cell_Type"].value_counts() 

### 1-) `Tool Setup`

In [ ]:
outlier_tool = access_data_path(f"{process_barcode}/class/{folder_outlier}", "tool" + ".ipynb") 
%run $outlier_tool 

### 2-) `Convert`

In [ ]:
# We need to convert the data to numpy arrays
Embeddings       = np.stack(data_full['Embeddings'].values) 
# However, as there will be subject_data_full after the first process, this expression sample is just for demonstration. Besides it needs to be reshaped in future. 
#Expression_Matrix                             = data_full['Expression'].values 
# BERT does not have second matrix 
#label_embeddings_matrix        = np.stack(data_full['Cell_Type_Embeddings'].values)   

In [ ]:
data_full

---
# 3- `Bench`

### 1-) `Embeddings`

#### 0-) `Oveview`

In [ ]:
data_full

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA


#### 1-) `Operation`

In [ ]:
# Extract embeddings
Embeddings = np.array(data_full['Embeddings'].tolist())

# Set the outlier detection method
subject_outlier = subject_outlier_0

# Temporarily set target_embeddings_matrix
target_embeddings_matrix = Embeddings.copy()

# Prepare input data
data_embedded = data_full.copy()

# Run the outlier detection script
# Ensure that `operation.ipynb` runs without issues and updates `subject_data_full`
outlier_operation = access_data_path(f"{process_barcode}/class/{folder_outlier}", "operation" + ".ipynb") 
%run $outlier_operation 

# Update data_full with the cleaned data
data_full = data_embedded.copy()
cleaned_data_full = subject_data_full.copy()  # This should now have outliers removed

#### 3-) `Report`

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Define a function for PCA and visualization
def visualize_embeddings(original_embeddings, cleaned_embeddings, title_suffix, switch_plot = 0):
    # Flatten the embeddings for easier visualization if necessary
    original_embeddings_flat = original_embeddings.reshape(-1, original_embeddings.shape[-1])
    cleaned_embeddings_flat = cleaned_embeddings.reshape(-1, cleaned_embeddings.shape[-1])
    
    # Apply PCA to reduce to 2D
    pca = PCA(n_components=2)
    original_embeddings_2d = pca.fit_transform(original_embeddings_flat)
    cleaned_embeddings_2d = pca.transform(cleaned_embeddings_flat)
    
    if switch_plot == 0: 
        # Plot original embeddings
        plt.figure(figsize=(12, 6))
        plt.scatter(original_embeddings_2d[:, 0], original_embeddings_2d[:, 1], alpha=0.5, label='Before Outlier Removal')
        plt.title(f'PCA of Embeddings Before Outlier Removal {title_suffix}')
        plt.xlabel('PCA Component 1')
        plt.ylabel('PCA Component 2')
        plt.legend()
        plt.show()
    elif switch_plot == 1:
        # Plot cleaned embeddings
        plt.figure(figsize=(12, 6))
        plt.scatter(cleaned_embeddings_2d[:, 0], cleaned_embeddings_2d[:, 1], alpha=0.5, label='After Outlier Removal', color='red')
        plt.title(f'PCA of Embeddings After Outlier Removal {title_suffix}')
        plt.xlabel('PCA Component 1')
        plt.ylabel('PCA Component 2')
        plt.legend()
        plt.show()
# Extract the embeddings from cleaned_data_full
cleaned_embeddings = np.array(cleaned_data_full['Embeddings'].tolist())


In [ ]:
visualize_embeddings(Embeddings, cleaned_embeddings, 'using Z-Score', switch_plot = 0)  

In [ ]:
# Visualize
visualize_embeddings(Embeddings, cleaned_embeddings, 'using Z-Score', switch_plot = 1)  

In [ ]:
print(f"Shape (Before): {data_full.shape}") 
print(f"Shape (After): {subject_data_full.shape}")  
print(f"Discarded rows: {data_full.shape[0] - subject_data_full.shape[0]}")  

print(f"Number of unique cell types: {len(pd.unique(data_full['Cell_Type']))}")  
print(f"Number of unique cell types: {len(pd.unique(subject_data_full['Cell_Type']))}")  
print(f"Lost cell types: {set(pd.unique(data_full['Cell_Type'])) - set(pd.unique(subject_data_full['Cell_Type']))}") 


### 2-) `Cell_Type_Gene_Marker_Embeddings`

#### 0-) `Oveview`

In [ ]:
data_full

#### 1-) `Operation`

In [ ]:
try:
    Expression_Matrix          = subject_data_full['Expression'].values 

    Expression_Matrix_Reshaped = Expression_Matrix.reshape(-1, 1) # Necessary for the pipeline  
except: 
    pass 

In [ ]:
try: 
    # Temporarly for operation 
    target_embeddings_matrix = Expression_Matrix_Reshaped.copy() 
    subject_outlier          = subject_outlier_1

    # Input data: data_full
    data_embedded = subject_data_full.copy() 
    outlier_operation = access_data_path(f"{process_barcode}/class/{folder_outlier}", "operation" + ".ipynb") 
    %run $outlier_operation 
    data_full = data_embedded.copy() 
    # Output: subject_data_full 
except:
    pass 

#### 2-) `Report`

In [ ]:
try: 
    print(f"Shape (Before): {data_embedded.shape}") 
    print(f"Shape (After): {subject_data_full.shape}")  
    print(f"Discarded rows: {data_embedded.shape[0] - subject_data_full.shape[0]}")  

    print(f"Number of unique cell types: {len(pd.unique(data_embedded['Cell_Type']))}")  
    print(f"Number of unique cell types: {len(pd.unique(subject_data_full['Cell_Type']))}")  
    print(f"Lost cell types: {set(pd.unique(data_embedded['Cell_Type'])) - set(pd.unique(subject_data_full['Cell_Type']))}") 
except:
    pass 

In [ ]:
subject_data_full

In [ ]:
subject_data_full["Cell_Type"].value_counts() 

---
# 4- `End`

In [ ]:
print(f"Subject Outlier 0 (Embedding): {subject_outlier_0}")
print(f"Subject Outlier 1 (Expression): {subject_outlier_1}")

In [ ]:
export_name = "data_raw" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_outlier}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(data_raw, file)   

export_name = "subject_data_full" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_outlier}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(subject_data_full, file)   
    
export_name = "subject_outlier_0" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_outlier}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(subject_outlier_0, file)   

export_name = "subject_outlier_1" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_outlier}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(subject_outlier_1, file)   